In [1]:
import csv
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer


In [2]:
from collections import Counter

# train은 업샘플링 한거, test은 안한 채로 만든 데이터 셋 불러옴

In [3]:
pre_train = pd.read_csv('good_pretrain.csv')
pre_test = pd.read_csv('final_test.csv')

In [4]:
pre_train

,Unnamed: 0,M_SAD,SitTime,PA_TOT,I_SB_EX,I_SB_FR,M_SUI_CON,i_SB_BK,BMI,M_STR,F_SODA,PA_MSC
0,0,M_SAD_2,SitTime_3,PA_TOT_5,I_SB_EX_5,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_4,BMI_1,M_STR_1,F_SODA_4,PA_MSC_4
1,1,M_SAD_1,SitTime_0,PA_TOT_4,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_1,i_SB_BK_2,BMI_1,M_STR_4,F_SODA_2,PA_MSC_6
2,2,M_SAD_1,SitTime_3,PA_TOT_2,I_SB_EX_5,I_SB_FR_5,M_SUI_CON_1,i_SB_BK_5,BMI_2,M_STR_3,F_SODA_1,PA_MSC_1
3,3,M_SAD_1,SitTime_0,PA_TOT_1,I_SB_EX_1,I_SB_FR_1,M_SUI_CON_1,i_SB_BK_1,BMI_1,M_STR_3,F_SODA_1,PA_MSC_1
4,4,M_SAD_1,SitTime_1,PA_TOT_1,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_1,i_SB_BK_2,BMI_1,M_STR_4,F_SODA_3,PA_MSC_1
...,...,...,...,...,...,...,...,...,...,...,...,...
59347,59347,M_SAD_2,SitTime_2,PA_TOT_2,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_1,i_SB_BK_2,BMI_2,M_STR_2,F_SODA_3,PA_MSC_3
59348,59348,M_SAD_1,SitTime_4,PA_TOT_1,I_SB_EX_5,I_SB_FR_5,M_SUI_CON_1,i_SB_BK_5,BMI_1,M_STR_4,F_SODA_1,PA_MSC_1
59349,59349,M_SAD_2,SitTime_0,PA_TOT_1,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_4,BMI_2,M_STR_2,F_SODA_2,PA_MSC_3
59350,59350,M_SAD_1,SitTime_1,PA_TOT_1,I_SB_EX_2,I_SB_FR_2,M_SUI_CON_2,i_SB_BK_2,BMI_1,M_STR_3,F_SODA_5,PA_MSC_6


In [5]:
pre_train = pre_train.drop(columns = ['Unnamed: 0'])
pre_test = pre_test.drop(columns = ['Unnamed: 0'])

# cbow모델에 넣기 좋게 리스트 형식으로 변환

In [6]:
train_list = pre_train.values.tolist()
test_list = pre_test.values.tolist()

In [7]:
len(train_list)

59352

<b> Tokenizing

In [8]:
def tokenize(corpus):
    """
    Tokenize the corpus of text.
    :param corpus: list containing a string of text (example: ["I like playing football with my friends"])
    :return corpus_tokenized: indexed list of words in the corpus, in the same order as the original corpus (the example above would return [[1, 2, 3, 4]])
    :return V: size of vocabulary
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    corpus_tokenized = tokenizer.texts_to_sequences(corpus)
    V = len(tokenizer.word_index)
    return corpus_tokenized, V

In [9]:
cor, V = tokenize(train_list)
V

58

In [10]:
cor_te, V_te = tokenize(test_list)
V_te

58

# 모델 훈련 sg = 1이라 지금은 skip gram

In [11]:
model_cbow = Word2Vec(train_list, workers = 6, size = 150 , min_count = 1, window = 11, sg = 0)

## Vocab들 모으기 (test data)

In [12]:
#create Vocab dict
cols = []
for temp in pre_test.columns:
    target = pre_test[temp].unique()
    for tesla in target:
        cols.append(tesla)

In [13]:
len(cols)

58

## Vocab 들을 각각 M_SUI_CON_1, M_SUI_CON_2와 유클리디안 거리를 구해. 

센트로이드는 M_SUI_CON_1, M_SUI_CON_2
Vocab들을 전부 센트로이드들이랑 유클리디안 거리를 구하고, 둘 중 가까운 쪽에다가 할당

In [14]:
def vec_means(cols, model):
    c1 = model.wv['M_SUI_CON_1']
    c2 = model.wv['M_SUI_CON_2']
    c1_cluster = []
    c2_cluster = []
    for col in cols:
        temp = []
        if np.linalg.norm(c1 - model.wv[col]) > np.linalg.norm(c2 - model.wv[col]): #거리가 더 짧은 센트로이드에 할당이므로
            temp.append(col)
            temp.append(np.linalg.norm(c2 - model.wv[col]))
            c2_cluster.append(temp)
        else:
            temp.append(col)
            temp.append(np.linalg.norm(c1 - model.wv[col]))
            c1_cluster.append(temp)
            
    return c1_cluster, c2_cluster
    

In [15]:
c1_c , c2_c = vec_means(cols, model_cbow)

In [16]:
c1_c = pd.DataFrame(c1_c)
c2_c = pd.DataFrame(c2_c)

In [17]:
c1_c.columns = ['Variables_con1', 'Euclidean distance_1']
c2_c.columns = ['Variables_con2', 'Euclidean distance_2']

In [18]:
c1_c = c1_c.sort_values(by = 'Euclidean distance_1')
c2_c = c2_c.sort_values(by = 'Euclidean distance_2')

In [19]:
c1_c.reset_index(drop = True, inplace = True)

In [20]:
c2_c.reset_index(drop = True, inplace = True)

In [23]:
con = pd.concat([c1_c, c2_c], axis = 1)

In [24]:
con

,Variables_con1,Euclidean distance_1,Variables_con2,Euclidean distance_2
0,M_SUI_CON_1,0.000000,M_SUI_CON_2,0.000000
1,SitTime_8,6.958497,F_SODA_6,8.450767
2,SitTime_7,7.465953,M_SAD_2,8.799477
3,BMI_4,7.782281,SitTime_6,9.127859
4,M_SAD_1,8.448430,F_SODA_5,9.483040
5,F_SODA_7,8.587315,SitTime_5,9.663481
6,BMI_3,9.232615,F_SODA_1,9.886292
7,F_SODA_4,9.580369,M_STR_1,9.974807
8,M_STR_5,9.656476,SitTime_4,9.982731
9,SitTime_0,9.665445,M_STR_2,10.044171


In [25]:
con.to_csv('means_clustering_cbow.csv')